In [1]:
import torch
from torch.utils import data
import random
from collections import Counter
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm
import sys
import os
from joblib import dump, load
from sparse_vector.sparse_vector import SparseVector
import time
from sklearn.metrics import roc_auc_score, f1_score
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections
from transformers import utils
from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from collections import defaultdict
from dna_tokenizer import DNATokenizer, seq2kmer
import logging
logging.disable(logging.WARNING)

2023-04-17 10:11:53.708991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 10:11:53.897813: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-17 10:11:55.250756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-17 10:11:55.250891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [2]:
class PredDataset(data.Dataset):
    def __init__(self, chroms, dna_source, intervals, tokenizer):

        self.chroms = chroms
        self.intervals = intervals
        self.tokenizer = tokenizer
        self.dna_source = dna_source
        
        
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = interval[1]
        end = interval[2]

        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.LongTensor(encoded_k_mers), (chrom, begin, end)

In [4]:
width = 128
pad = 192
k_mer_pad = 5

def final_prediction(chrom, DNA, models, device):
    
    intervals = []
    ends = []
    
    
    prediction = np.zeros(len(DNA[chrom]), dtype=np.float32)
    
    
    for st in range(0, len(DNA[chrom]) - 512, width):
        interval = [st, min(st + 512, len(DNA[chrom]))]
        intervals.append([chrom, interval[0], interval[1]])
        
    pred_dataset = PredDataset(chroms, DNA, intervals, 
                               DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))

    params = {'batch_size':32, 'num_workers':5, 'shuffle':False}
    load_predict = data.DataLoader(pred_dataset, **params)

    
    for model_i, model in enumerate(models):
    
        model.to(device)
        with torch.no_grad():
            for input_ids, intervals in tqdm(load_predict):
                input_ids = input_ids.to(device)
                outputs = torch.softmax(model(input_ids = input_ids)['logits'],axis = -1).cpu().numpy()[:,:,1]
                for ind, interval in enumerate(zip(intervals[0], intervals[1], intervals[2])): 
                    if interval[1] == 0:
                        prediction[interval[1]:interval[2]] = outputs[ind]
                    else:    
                        prediction[interval[1]+pad:interval[2]] = outputs[ind, pad:]
                    
        dump(prediction, f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_{model_i}_{chrom}', 3)

In [7]:
hg19_chrom_sizes = pd.read_csv('data/hg19.chrom.sizes', header = None, sep = '\t', index_col=0).to_dict()[1]
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]

G4 = {}
for chrom in chroms:
    G4[chrom] = np.zeros(hg19_chrom_sizes[chrom], dtype = bool)
       
with open("data/Raji_ssDNA_enriched_Quadruplex.bed")as f:
    for idx, line in enumerate(f):
        if idx>0:
            chrom, start, end, _ , _ , _ = line.strip().split()
            if chrom in G4:
                G4[chrom][int(start):int(end)] = 1

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"data/hg19_dna/{file}") for file in files])

DNA = {chrom:chrom_reader(chrom) for chrom in chroms}

In [8]:
models = []

for MODEL_NUMBER in range(5):
    dir_to_pretrained_model = f'dnabert_hg_fold_{MODEL_NUMBER}_kouzine_g4'
    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model)
    model.eval()
    models.append(model)

In [ ]:
for chrom in chroms[:5]:
    print(f"BEGIN CHROM {chrom}")
    final_prediction(chrom, DNA, models, device = 0)

BEGIN CHROM chr1


 25%|██████████████████████████████████████▌                                                                                                                 | 15417/60853 [1:16:39<3:45:39,  3.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 52%|███████████████████████████████████████████████████████████████████████████████▎                                                                        | 31775/60853 [2:37:49<2:23:22,  3.38it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs

BEGIN CHROM chr3


  2%|███▏                                                                                                                                                       | 1009/48346 [05:00<3:54:59,  3.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  8%|████████████▉                                                                                                                                              | 4031/48346 [20:00<3:40:01,  3.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs

BEGIN CHROM chr5


 86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                     | 38082/44169 [3:10:27<30:21,  3.34it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 25%|██████████████████████████████████████▉                                                                                                                   | 11174/44169 [56:42<2:46:51,  3.30it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs

In [10]:
equalized, divisions = load('data/hg_divisions_kouzine.pkl')

In [11]:
com_len = sum([len(DNA[chrom]) for chrom in chroms])
sums = []

for chrom in tqdm(chroms):
    loc_sum = []
    for model_num in range(5):
        vec = load(f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_{model_num}_{chrom}')
        loc_sum.append(vec.sum())
    sums.append(loc_sum)

multipliers = np.array(sums).sum(axis=0) / com_len

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [06:03<00:00, 15.16s/it]


In [12]:
for chrom in tqdm(chroms):
    vecs = np.array([load(f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_{model_num}_{chrom}') 
                     for model_num in range(5)])
    res_vec = (vecs / multipliers[:, None]) * multipliers.mean()
    mean_vec = res_vec.mean(axis=0)
    
    test_ints = []
    for MODEL_NUMBER in range(5):
        train_inds, test_inds = divisions[MODEL_NUMBER]
        train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]
        test_ints.extend([(MODEL_NUMBER, inter) for inter in test_intervals if inter[0] == chrom])
    
    for model_num, inters in test_ints:
        mean_vec[inters[1]: inters[2]] = res_vec[model_num, inters[1]: inters[2]]
    dump(mean_vec, f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_res_{chrom}', 3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [21:42<00:00, 54.26s/it]


In [14]:
all_pred = []
all_true = []
for chrom in tqdm(chroms):
    true_clean = G4[chrom][:].astype(int)
    all_pred.append(load(f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_res_{chrom}'))
    all_true.append(true_clean)
    
print(roc_auc_score(np.concatenate(all_true), np.concatenate(all_pred)))
print(sklearn.metrics.classification_report(np.concatenate(all_true), np.concatenate(all_pred)>0.5, digits=4))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:09<00:00,  7.90s/it]


0.999727218746237
              precision    recall  f1-score   support

           0     1.0000    0.9991    0.9996 3094121023
           1     0.3571    0.9630    0.5210   1556389

    accuracy                         0.9991 3095677412
   macro avg     0.6785    0.9811    0.7603 3095677412
weighted avg     0.9997    0.9991    0.9993 3095677412



In [15]:
from tabulate import tabulate
def to_fwf(df, fname):
    content = tabulate(df.values.tolist(), tablefmt="plain")
    open(fname, "w").write(content)

pd.DataFrame.to_fwf = to_fwf

In [16]:
pchroms, starts, ends = [], [], []
model_confidence_threshold = 0.25
min_length = 6

chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]

for chrom in tqdm(chroms):
    pred = load(f'/gim/lv01/dumerenkov/HG19_G4/HG19_kouzine_res_{chrom}')
    labeled, max_label = scipy.ndimage.label(pred>model_confidence_threshold)
    for idx in range(1,max_label+1):
        where = np.where(labeled == idx)[0]
        start = where[0]
        end = where[-1] + 1
        
        if end-start>min_length:
            pchroms.append(chrom)
            starts.append(start)
            ends.append(end)
pd.DataFrame(list(zip(pchroms, starts, ends))).to_fwf(f'beds/HG19_thr_{model_confidence_threshold}_minlen_{min_length}.bed')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [10:17:35<00:00, 1543.99s/it]
